In [ ]:
import numpy as np

import torch
import torchvision # здесь находятся различные популярные датасеты

if torch.cuda.is_available():
    print("CUDA available")
    torch.cuda.manual_seed(0)
else:
    print("No CUDA for you :(")
    
print(torch.__version__)
%config InlineBackend.figure_format ='retina' # чтобы графики были красивее
%matplotlib inline

# Зачем нужен pytorch?

1) Замена ```numpy```  свозможностью работы на GPU 

2) Низкоуровневая библиотека для ```DEEP LEARNING```

# Базовые операции

Основным элементом ```pytorch``` являются __тензоры__ - мы будем воспринимать их как н-мерные массивы.<br> 
Пример:

In [ ]:
torch.zeros(5,4)

In [ ]:
torch.rand(5,4)

Создавать тензоры можно и вручную из списков и из массивов:

In [ ]:
print(torch.tensor([1,2,3]))
print(torch.tensor(np.array([1,2,3])))

Также можно тензоры обратно переводить в массивы ```numpy``` за константное время:

In [ ]:
x = torch.rand(10,10)
%timeit x.numpy
x = torch.rand(100,100)
%timeit x.numpy
x = torch.rand(1000,1000)
%timeit x.numpy

Точно так же, как и в ```numpy```, можно производить арифметические операции над тензорами:

In [ ]:
x = torch.ones(5,3)
y = torch.rand(5, 3)

In [ ]:
print(x)
print(y)

In [ ]:
print(x+y)
print(torch.add(x,y))

In [ ]:
print(x*y)
print(torch.mul(x,y))

Операции, заканчивающиеся на нижнее подчеркивание, __не чистые__, то есть они изменяют элемент, к которому были применены. <br> Пример:

In [ ]:
print(y)
y.add_(x)
print(y)

В ```pytorch``` поддерживаются вычисления на видеокарте:

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")          
    y = torch.ones_like(x, device=device)  
    x = x.to(device)                       
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))     
else:
    print("FeelsBadMan")

# AUTOGRAD

Пакет ```autograd``` - одна из самых важных вещей в ```pytorch```. В нем находятся функции для автоматического взятия производных от тензоров. 

У ```torch.tensor``` есть аттрибут ```.requires_grad``` , и если его поставить ```True``` , то все операции над этим тензором начнут записываться, чтобы можно было взять производную.

In [ ]:
x = torch.ones(2,2, requires_grad = True)

In [ ]:
x

Чтобы остановить отслеживание операций, к тензору надо применить метод ```.detach()```

Еще один важное понятие  для ```autograd``` - это ```.grad_fn```<br>
Это аттрибут тензора, который показывает, с помощью какой функции был создан этот тензор.<br> Если тензор был создан вручную, то ```.grad_fn = None```

In [ ]:
print(x.grad_fn)

In [ ]:
y = x + 2
y

In [ ]:
print(y.grad_fn)

Чтобы посчитать производную тензора, к нему применяется метод ```.baсkward()```.

Если тензор состоял из единственного числа, то в ```.backward``` никаких аргументов передавать не нужно. <br>
[Подробнее об аргументах ```.backward``` - второй ответ](https://stackoverflow.com/questions/43451125/pytorch-what-are-the-gradient-arguments)

Создадим тензор с ```.requires_grad = True``` для отслеживания вычислений над ним, и совершим несколько операций:

In [ ]:
x = torch.ones(2,2,requires_grad = True)
x

In [ ]:
y = x + 2

In [ ]:
z = 3 * y**2
z.requires_grad_(True)

In [ ]:
out = z.mean()
out

In [ ]:
out.backward(torch.tensor([1.]))

In [ ]:
x.grad

Объяснение произошедшему:

$$
\begin{align}
\textbf{out} = \frac14 \sum_i{z_i} = \frac14 \sum_i{3~(x_i+2)^2} \newline
\newline
\frac{\partial ~\textbf{out}}{\partial ~x_i} = \frac{\partial ~out}{\partial ~z} \frac{\partial ~z}{\partial ~y} \frac{\partial ~y}{\partial ~x_i} 
\newline
\newline
\frac{\partial ~\textbf{out}}{\partial ~x_i} =  \frac32~(x_i+2) = \frac92 = 4.5
\end{align}
$$

<div class="alert alert-warning">

<b>Задание</b>

```x = torch.eye(2, requires_grad = True)
y = x/12
z = (y + 5)**2
out = z.sum()```
<br>
<br>
Что вернет ```x.grad``` после вызова ```out.backward()```?
</div>

# Нейронные сети

Построим модель нейронной сети, в которой мы сами реализуем разбиение данных на мини-батчи, итерации по ним и прямое распространение(```forward propagation```). <br>
Обратное распространение (```backpropagation```) сделаем при помощи ```.backward```

In [ ]:
import time
from sklearn.datasets import make_moons # генератор данных 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
xm, ym = make_moons(n_samples = 10000, noise=0.1)
ym = ym.reshape(-1,1)
xm_train, xm_test, ym_train, ym_test = [torch.tensor(i) for i in train_test_split(xm,ym, test_size = 0.2)]

Генератор минибатчей:

In [ ]:
def generate_minibatches(inputs, targets, batchsize = 16, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_id in range(0, inputs.shape[0], batchsize):
        end_id = min(start_id + batchsize, inputs.shape[0])
        if shuffle:
            batch_indices = indices[start_id:end_id]
        else:
            batch_indices = slice(start_id, end_id)
        yield inputs[batch_indices], targets[batch_indices]

В ```pytorch``` модель состоит из слоев, например ```nn.Linear(...)```<br>
```nn.Linear(...)``` - набор параметров (матрица весов и вектор смещения)

In [ ]:
list(nn.Linear(2,20).parameters())

Чтобы задать модель нейронной сети, надо определить методы ```__init__```, в котором определяется структура нейронной сети, и ```forward```, где описывается прямое распространение. <br>
А также модель должна наследовать класс ```nn.Module```

## Создание

In [ ]:
class Net(nn.Module):

    def __init__(self, seed = 0):
        super().__init__()
            
        self.l1 = nn.Linear(2, 20)
        self.l2 = nn.Linear(20, 1)
        
        
    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.sigmoid(self.l2(x))
        
        return x

net = Net()
print(net)

В ```pytorch``` представлены несколько видов функций потерь, например: <br>
```nn.MSELoss()``` - Mean Squared Error <br>
```nn.L1Loss()``` - MAE <br>
```nn.CrossEntropyLoss()``` - перекрестная энтропия, в которую входит softmax<br>
То есть, если мы хотим использовать ```nn.CrossEntropyLoss()``` в качестве функции потерь в модели нейронной сети, на последнем слое __не надо применять softmax__. Также особенностью реализации ```nn.CrossEntropyLoss()``` в ```pytorch``` является то, что эта функция потерь принимает на вход __не one-hot-encoded вектора__, а обычные номера классов.
    
[Полный список функций потерь](https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html)

## Обучение

В этом примере мы взяли на себя итерирование по эпохам, минибатчам, подсчет функционала качества, обновление значений весов и смещений и обнуление градиентов тензоров после обновления.

### Зачем обнулять градиенты?
Каждый раз, когда вызывается метод ```.backward``` градиенты тензоров не обновляются (то есть их __не замещают новые значения__), а __накапливаются__. Поэтому, чтобы градиенты не смешивались, необходимо на каждом шаге их обнулять.

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed) # это нужно, так как генератор минибатчей использует np.random.shuffle


net1 = Net()

criterion = nn.MSELoss()

lr = 0.1

for epoch in range(100):
    
    running_loss = 0.0
    
    for batch in generate_minibatches(xm_train,ym_train,64,shuffle = True):
        
        data, labels = batch
        
        # Прямое распространение
        output = net1(data.float())
        
        # Подсчет ошибки на батче
        loss = criterion(output, labels.float())
        
        # Обратное распространение
        loss.backward()
    
        #Обновление весов и обнуление градиентов
        with torch.no_grad(): # torch.no_grad() отключает отслеживание вычислений над тензорами в блоке
            for parameter in net1.parameters():
                parameter.data -= lr * parameter.grad
                parameter.grad = torch.zeros_like(parameter)

        running_loss += loss.item()
            
            
    if epoch%10==0:
        print(f"Loss on epoch {epoch} - {running_loss}") 
        

Того же самого результата можно добиться, используя встроенные в ```pytorch``` оптимизаторы и функции для обновления и обнуления градиентов.

Виды возможных оптимизаторов:<br>
```optim.SGD``` - стохастический градиентный градиентный спуск (на самом деле, ему можно подавать батч любого размера, так что SGD - градиентный спуск по минибатчам)<br>
```optim.Adam```  [ссылка на курсеру](https://www.coursera.org/learn/deep-neural-network/lecture/w9VCZ/adam-optimization-algorithm)<br>
```optim.RMSprop```  [ссылка на курсеру](https://www.coursera.org/learn/deep-neural-network/lecture/BhJlm/rmsprop) <br>
<br>
[Полный список оптимизаторов](https://pytorch.org/docs/stable/optim.html])

Чтобы инициализировать оптимизатор, в него надо передать параметры обучаемой модели ```net.parameters()``` и начальную скорость обучения (learning rate)

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
    
net2 = Net()

# В нашем случае будем использовать обычный градиентный спуск по минибатчам, как и в прошлый раз
optimizer = optim.SGD(net2.parameters(), lr = 0.1)

criterion = nn.MSELoss()

for epoch in range(100):
    running_loss = 0.0
    for batch in generate_minibatches(xm_train,ym_train,64,shuffle = True):
        
        data, labels = batch
        
        # Прямое распространение
        output = net2(data.float())
        
        # Подсчет ошибки на батче
        loss = criterion(output, labels.float())
        
        # Обратное распространение
        loss.backward()
        
        #Обновление весов
        optimizer.step()
        
        #Обнуление градиентов
        optimizer.zero_grad() 
        
        running_loss += loss.item()
            
    if epoch%10==0:
        print(f"Loss on epoch {epoch} - {running_loss}") 

Посмотрим, что получилось:

In [ ]:
def plot_labels(data,target):
    df = pd.DataFrame(dict(x=data[:,0], y=data[:,1], label=target))
    colors = {0:'red', 1:'blue'}
    fig, ax = plt.subplots()
    grouped = df.groupby('label')
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_labels(xm_test.numpy(),np.round(net2(xm_test.float()).detach()).reshape(-1,))

# Вычисление медианной стоимости дома

Как и раньше стоит задача предсказания медианной стоимости дома.
Подгрузим данные и разобьем их на __train__ и __val__.

In [ ]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe.head()


In [ ]:
def preprocess_features(df):
    """Prepares input features from California housing data set.

    Args:
      california_housing_dataframe: A Pandas DataFrame expected to contain data
        from the California housing data set.
    Returns:
      A DataFrame that contains the features to be used for the model, including
      synthetic features.
    """
    selected_features = df[
            ["latitude",
             "longitude",
             "housing_median_age",
             "total_rooms",
             "total_bedrooms",
             "population",
             "households",
             "median_income"]]
    processed_features = selected_features.copy()
    # Create a synthetic feature.
    processed_features["rooms_per_person"] = (df["total_rooms"]/df["population"])
    return processed_features

def preprocess_targets(df):
    """Prepares target features (i.e., labels) from California housing data set.

      Args:
        california_housing_dataframe: A Pandas DataFrame expected to contain data
          from the California housing data set.
      Returns:
        A DataFrame that contains the target feature.
      """
    output_targets = pd.DataFrame()
    # Scale the target to be in units of thousands of dollars.
    output_targets["median_house_value"] = (df["median_house_value"] / 1000.0)
    return output_targets

In [ ]:
# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

Переведем все в ```torch.tensor```.

In [ ]:
training_examples = torch.tensor(training_examples.values).float()
training_targets = torch.tensor(training_targets.values).float()

validation_examples = torch.tensor(validation_examples.values).float()
validation_targets = torch.tensor(validation_targets.values).float()

Надо предсказать единственное вещественное значение, а количество признаков - 9, а значит на входном слое сети будет 9 нейронов, а на выходном 1.

Зададим свою сеть со скрытыми слоями.

In [ ]:
class HouseValueNet(nn.Module):

    def __init__(self, seed = 0):
        super().__init__()
            
        self.l1 = nn.Linear(9, 20)
        self.l2 = nn.Linear(20, 10)
        self.l3 = nn.Linear(10, 1)
        with torch.no_grad(): # torch.no_grad() отключает отслеживание вычислений над тензорами в блоке
            for parameter in self.parameters():
                parameter.data*=0.01
          
    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        x = self.l3(x)
        
        return x

net = HouseValueNet()
print(net)

Обучим нейронную сеть. В качестве оптимизатора возьмем ```optim.Adam``` с ```lr = 0.05```.

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
    
net3 = HouseValueNet()

# Будем использовать обычный градиентный спуск по минибатчам, как и в прошлый раз
optimizer = optim.Adam(net3.parameters(),  lr = 0.05)

criterion = nn.MSELoss()
losses = []
for epoch in range(100):
    running_loss = 0.0
    batch_counter = 0
    for batch in generate_minibatches(training_examples,training_targets,64,shuffle = True):
        
        data, labels = batch
        
        # Прямое распространение
        output = net3(data.float())
        
        # Подсчет ошибки на батче
        loss = torch.sqrt(criterion(output, labels.float()))

        # Обратное распространение
        loss.backward()
        
        #Обновление весов
        optimizer.step()
        
        #Обнуление градиентов
        optimizer.zero_grad() 
        
        running_loss += loss.item()
        losses.append(loss)
        batch_counter +=1
        
    if epoch%10==0:
        print(f"Mean loss on epoch {epoch} - {running_loss/batch_counter}") 

Проверим качество полученной модели на отложенной выборке:

In [ ]:
torch.sqrt(criterion(net3(validation_examples), validation_targets))

# MNIST

Обучим нейронную сеть на датасете  MNIST, содержащем 10 видов различной одежды:

In [ ]:
from keras.datasets import mnist, fashion_mnist, cifar10
from keras.utils.np_utils import to_categorical  
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train/255
x_test = x_test/255

ycat_train = to_categorical(y_train, num_classes=10)
ycat_test = to_categorical(y_test, num_classes=10)

In [ ]:
plt.imshow(x_train[0],cmap = 'gray')

In [ ]:
ycat_train = torch.tensor(ycat_train)
ycat_test = torch.tensor(ycat_test)

x_train = torch.tensor(x_train).view(-1,28*28).float()
y_train = torch.tensor(y_train).float()
x_test = torch.tensor(x_test).view(-1,28*28).float()
y_test = torch.tensor(y_test).float()


In [ ]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

Построим модель с ```nn.CrossEntropyLoss()```. На последнем слое не будем применять ```torch.softmax```, так как в ```nn.CrossEntropyLoss()``` эта функция уже применяется. <br>
Но чтобы получить нормальные предсказания в виде вектора вероятностей быть той или иной цифрой и проверить точность предсказаний, применить ```torch.softmax``` все же придется.

In [ ]:
class MnistSoftmaxNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x.float()))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

net = MnistSoftmaxNet()
print(net)

In [ ]:
#!pip install tqdm

from tqdm import trange

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
    
net = MnistSoftmaxNet()


optimizer = optim.Adam(net.parameters())

criterion = nn.CrossEntropyLoss()

for epoch in trange(5):
    running_loss = 0.0
    batch_counter = 0
    for batch in generate_minibatches(x_train, y_train, 64, shuffle = True):
        
        data, labels = batch
        
        # Прямое распространение
        output = net(data)
        
        # Подсчет ошибки на батче
        loss = criterion(output, labels.long()) 
        
        # Обратное распространение
        loss.backward()
        
        #Обновление весов
        optimizer.step()
        
        #Обнуление градиентов
        optimizer.zero_grad() 
        
        running_loss += loss.item()
        
        batch_counter+=1
    
    if epoch%1==0:
        print(f"Mean loss on epoch {epoch} - {running_loss/batch_counter}") 

Получили довольно высокую точность на тестовой выборке:

In [ ]:
accuracy_score(torch.argmax(torch.softmax(net(x_test.float()),-1),dim = -1), y_test)

На самом деле, создавать модели можно и более простым способом. Но тогда теряется гибкость настройки модели и мы лишаемся возможности создавать свои слои:

In [ ]:
layers = []
layers.append(nn.Linear(784, 128))
layers.append(nn.ReLU())
layers.append(nn.Linear(128, 64))
layers.append(nn.ReLU())
layers.append(nn.Linear(64, 10))

net = nn.Sequential(*layers)

In [ ]:
net

In [ ]:
#!pip install tqdm

from tqdm import trange

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)


optimizer = optim.Adam(net.parameters())

criterion = nn.CrossEntropyLoss()

for epoch in trange(5):
    running_loss = 0.0
    batch_counter = 0
    for batch in generate_minibatches(x_train, y_train, 64, shuffle = True):
        
        data, labels = batch
        
        # Прямое распространение
        output = net(data)
        
        # Подсчет ошибки на батче
        loss = criterion(output, labels.long()) 
        
        # Обратное распространение
        loss.backward()
        
        #Обновление весов
        optimizer.step()
        
        #Обнуление градиентов
        optimizer.zero_grad() 
        
        running_loss += loss #вызов .item() переводит полученное число на cpu, что замедляет работу
        
        batch_counter+=1
        
    if epoch%1==0:
        print(f"Loss on epoch {epoch} - {running_loss/batch_counter}") 

In [ ]:
accuracy_score(torch.argmax(torch.softmax(net(x_test.float()),-1),dim = -1), y_test)

# Заключение

Мы рассмотрели базовые понятия ```pytorch``` и научились строить модели полносвязных нейронных сетей используя встроенные функции.